## view pandas / load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import json
from dateutil.parser import parse
import pytz
from sklearn import preprocessing, datasets, linear_model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, r2_score
import collections
import pickle

In [2]:
# Load the dataset
c_df = pd.read_json('NY_Concerts.json')
v_df = pd.read_json('NY_Venues.json')

# Consistent numbers
np.random.seed(0)

In [3]:
#Drop useless columns
c_df.dropna(subset=['average_price', 'date&time_event'], inplace=True)

# Replace string nan to numpy nan
c_df = c_df.replace('NaN', np.nan)

# Events Json to Pandas DF

In [4]:
training = c_df[c_df['ticket_listing_count']>3]
missing_genre_artists = training[training.performer_genre.isnull()]['performer_name'].unique()
missing_genres_artists = list(missing_genre_artists)
%store missing_genres_artists

Stored 'missing_genres_artists' (list)


In [5]:
c_df.head()

,announce_date,average_price,date&time_event,event_title,highest_price,lowest_price,median_price,performer_genre,performer_name,ticket_listing_count,type_event,upcoming_events?,venue_city,venue_name,venue_zipcode,visible_until_utc
0,2019-02-21T00:00:00,71.0,2019-05-28T19:00:00,Bobby,161.0,87.0,67.0,Pop,Bobby,5.0,concert,True,Buffalo,Sheas Performing Arts Center,14202,2019-05-29T03:00:00
1,2019-05-09T00:00:00,20.0,2019-05-28T19:00:00,The Heroes,20.0,20.0,20.0,NaN,The Heroes,1.0,concert,True,Brooklyn,The Kingsland,11222,2019-05-29T03:00:00
2,2019-02-21T00:00:00,245.0,2019-05-28T19:00:00,The Japanese House (18+),248.0,248.0,245.0,Alternative,The Japanese House,1.0,concert,True,New York,Webster Hall,10003,2019-05-29T03:00:00
4,2019-02-13T00:00:00,58.0,2019-05-28T19:00:00,Haruomi Hosono (16+),64.0,52.0,58.0,NaN,Haruomi Hosono,5.0,concert,True,New York,Gramercy Theatre,10010,2019-05-29T03:00:00
6,2019-04-20T00:00:00,20.0,2019-05-28T19:00:00,Michelle Walker,20.0,20.0,20.0,NaN,Michelle Walker,1.0,concert,True,New York,Rockwood Music Hall - Stage 3,10002,2019-05-29T03:00:00


In [6]:
# Resort values
c_df.reset_index(drop=True)

# Genres

In [7]:
#found all nec data
%store -r missing_artist_list

In [8]:
#drop all rows that have no genre
missing_artist_list = missing_artist_list[missing_artist_list.artist_genre != 'No Matches']

In [9]:
#drop duplicates
missing_artist_list.drop_duplicates(subset='artist_name', keep="last", inplace=True)

/Users/chrischung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
c_df['performer_genre'].isna().sum()

345

In [11]:
missing_artist_list.reset_index(drop=True, inplace=True)

In [13]:
c_df.performer_genre = c_df.performer_genre.combine_first(missing_artist_list.artist_genre)

In [14]:
# 7 hours of work for 27 genres

c_df['performer_genre'].isna().sum()

331

In [15]:
# c_df['performer_genre'] = c_df['performer_genre'].apply(lambda x: x.lower().replace('-', ''))

# Dates

In [16]:
# Convert string dates to datetime objects

In [17]:
c_df['announce_date'] = pd.to_datetime(c_df['announce_date']).dt.date
c_df['visible_until_utc'] = pd.to_datetime(c_df['visible_until_utc'])
# c_df['visible_until_utc'] = pd.to_datetime(c_df['visible_until_utc'])

#Changing Visible Until UTC to Eastern time
eastern = pytz.timezone('US/Eastern')
c_df['visible_until_est'] = c_df['visible_until_utc'].dt.tz_localize('UTC').dt.tz_convert(eastern)

#Convert event time to weekend/weekday and afternoon/evening

c_df['event_year']= [d.split('-')[0] for d in c_df['date&time_event']]
c_df['event_month']= [d.split('-')[1] for d in c_df['date&time_event']]
c_df['event_day']= [d.split('-')[2] for d in c_df['date&time_event']]

# Create 
c_df['date&time_event'] = pd.to_datetime(c_df['date&time_event'])
c_df['event_start'] = pd.to_datetime(c_df['date&time_event']).dt.strftime("%H")
c_df['event_date'] = pd.to_datetime(c_df['date&time_event']).dt.date
c_df['event_month'] = pd.to_datetime(c_df['event_date']).dt.strftime("%m")
c_df['event_day'] = pd.to_datetime(c_df['date&time_event']).dt.strftime("%a")

#Create ticket window column
c_df['ticket_avail_for'] = pd.to_datetime(c_df['visible_until_utc'].dt.date - c_df['announce_date']).dt.strftime("%d")



# Afternoon or Evening

In [18]:
type(c_df['event_start'][0])

str

In [19]:
# Change values to float
c_df['event_start'] = c_df['event_start'].apply(lambda x: int(x))

In [20]:
#drop all times before 11
before_11 = c_df[c_df['event_start']<12].index
c_df.drop(before_11, inplace=True)
c_df[c_df['event_start']<12]

#drop all times above 24
after_24 = c_df[c_df['event_start']>24].index
c_df.drop(after_24, inplace=True)
c_df[c_df['event_start'] >24]


,announce_date,average_price,date&time_event,event_title:,highest_price,lowest_price,median_price,performer_genre,performer_name,ticket_listing_count,...,venue_name,venue_zipcode,visible_until_utc,visible_until_est,event_year,event_month,event_day,event_start,event_date,ticket_avail_for


In [21]:
c_df.reset_index(inplace=True)

In [22]:
type(c_df.loc[c_df['event_start'][0]])

pandas.core.series.Series

In [23]:
c_df['event_start'].value_counts().sort_values()

16      2
12      5
15      7
14     10
23     13
17     14
13     43
22     51
18     90
21    132
19    479
20    866
Name: event_start, dtype: int64

In [24]:
def f(row):
    if (12 <= row['event_start'] <= 17):
        val = 'afternoon'
    elif (18 <= row['event_start'] <= 23):
        val = 'evening'
    return val

In [25]:
c_df['time_of_day'] = c_df.apply(f, axis=1)

In [26]:
c_df['time_of_day'].value_counts()

evening      1631
afternoon      81
Name: time_of_day, dtype: int64

# Weekend or Weekday

In [27]:
type(c_df['event_day'])

pandas.core.series.Series

In [28]:
c_df['event_day'].value_counts()

Sat    416
Fri    378
Thu    266
Sun    222
Wed    196
Tue    150
Mon     84
Name: event_day, dtype: int64

In [29]:
c_df['event_day'].apply(lambda x: str(x))

0       Fri
1       Fri
2       Fri
3       Fri
4       Fri
5       Fri
6       Fri
7       Fri
8       Fri
9       Fri
10      Fri
11      Fri
12      Fri
13      Fri
14      Fri
15      Fri
16      Sat
17      Sat
18      Sat
19      Sat
20      Sat
21      Sat
22      Sat
23      Sat
24      Sat
25      Sat
26      Sat
27      Sat
28      Sat
29      Sat
       ... 
1682    Wed
1683    Thu
1684    Fri
1685    Sat
1686    Sat
1687    Sun
1688    Wed
1689    Thu
1690    Sat
1691    Sun
1692    Sat
1693    Sun
1694    Sun
1695    Sat
1696    Sat
1697    Sun
1698    Sun
1699    Fri
1700    Sun
1701    Fri
1702    Sun
1703    Sun
1704    Sat
1705    Sun
1706    Thu
1707    Sat
1708    Sun
1709    Sun
1710    Sat
1711    Fri
Name: event_day, Length: 1712, dtype: object

In [30]:
def d(row):
    if (c_df['event_day'] == 'Sat').any():
        val = 'weekend'
    else:
        val = 'weekday'
    return val

In [31]:
c_df['wkend_wkday'] = c_df.apply(d, axis=1)

In [32]:
c_df['wkend_wkday'].value_counts()

weekend    1712
Name: wkend_wkday, dtype: int64

In [33]:
c_df.head()

,index,announce_date,average_price,date&time_event,event_title:,highest_price,lowest_price,median_price,performer_genre,performer_name,...,visible_until_utc,visible_until_est,event_year,event_month,event_day,event_start,event_date,ticket_avail_for,time_of_day,wkend_wkday
0,0,2018-10-13,58.0,2019-02-01 18:30:00,Young Nudy with SahBabii,63.0,54.0,54.0,Hip-Hop,SahBabii,...,2019-02-02 03:30:00,2019-02-01 22:30:00-05:00,2019,02,Fri,18,2019-02-01,23,evening,weekend
1,2,2018-10-16,114.0,2019-02-01 19:00:00,Umphrey's McGee with Robert Walter's 20th Cong...,117.0,111.0,111.0,Pop,Umphrey's McGee,...,2019-02-02 04:00:00,2019-02-01 23:00:00-05:00,2019,02,Fri,19,2019-02-01,20,evening,weekend
2,3,2018-11-27,337.0,2019-02-01 19:00:00,ZyanosE with Hank Wood & The Hammerheads,584.0,253.0,256.0,rock,Hank Wood & The Hammerheads,...,2019-02-02 04:00:00,2019-02-01 23:00:00-05:00,2019,02,Fri,19,2019-02-01,09,evening,weekend
3,8,2018-12-18,41.0,2019-02-01 19:00:00,Sighns with Cyberattack,46.0,36.0,36.0,blues,Sighns,...,2019-02-02 04:00:00,2019-02-01 23:00:00-05:00,2019,02,Fri,19,2019-02-01,16,evening,weekend
4,9,2018-12-20,337.0,2019-02-01 19:30:00,Ritual Talk with The Humble Cheaters,584.0,253.0,256.0,electronic,Ritual Talk,...,2019-02-02 04:30:00,2019-02-01 23:30:00-05:00,2019,02,Fri,19,2019-02-01,14,evening,weekend


# Dropping columns and Filling NaN values

In [34]:
c_df.drop(columns=['index','announce_date', 'visible_until_utc', 'visible_until_est',\
                  'event_year', 'event_date', 'event_title:', 'upcoming_events?', 'date&time_event',\
                  'type_event', 'venue_zipcode'],inplace=True)

# Drop values with ticket count under 3
under_3 = c_df[c_df['ticket_listing_count']<3].index
c_df.drop(under_3, inplace=True)

# Fill NaN values with average prices
c_df[c_df['median_price'].isna()]
c_df['median_price'].fillna(c_df['average_price'].isna(),inplace=True)
c_df['median_price'] = c_df['median_price'].apply(lambda x: float(x))

In [35]:
c_df = c_df.reset_index(drop=True)

In [36]:
#drop performers without genre
c_df.dropna(inplace=True)

In [37]:
# 1281 good to use data points
c_df =c_df.reset_index(drop=True)
c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281 entries, 0 to 1280
Data columns (total 15 columns):
average_price           1281 non-null float64
highest_price           1281 non-null float64
lowest_price            1281 non-null float64
median_price            1281 non-null float64
performer_genre         1281 non-null object
performer_name          1281 non-null object
ticket_listing_count    1281 non-null float64
venue_city              1281 non-null object
venue_name              1281 non-null object
event_month             1281 non-null object
event_day               1281 non-null object
event_start             1281 non-null int64
ticket_avail_for        1281 non-null object
time_of_day             1281 non-null object
wkend_wkday             1281 non-null object
dtypes: float64(5), int64(1), object(9)
memory usage: 150.2+ KB


# Venues

In [38]:
v_df.head()

,venue_capacity,venue_name,venue_score
0,250,The Cutting Room,0.573900
1,206,Baby's All Right,0.424032
2,0,Knitting Factory Brooklyn,0.468147
3,0,Brooklyn Bowl,0.527452
4,0,Iridium Jazz Club,0.487660


In [39]:
v_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2875 entries, 0 to 2874
Data columns (total 3 columns):
venue_capacity    2875 non-null int64
venue_name        2875 non-null object
venue_score       2875 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 67.5+ KB


In [40]:
#Drop duplicate venues
v_df.drop_duplicates(subset=['venue_name'],inplace=True)

In [41]:
v_df = v_df.reset_index(drop=True)

In [42]:
# Merge c_df and v_df dataframes 
c_df = pd.merge(c_df, v_df, how='left', on='venue_name').reset_index(drop=True)

In [43]:
# c_df.loc[c_df['performer_name'] == 'Pop Evil']['venue_capacity'] = 2500

In [55]:
c_df.loc[c_df['venue_capacity'] == 0]['venue_name'].unique()

array(['Knitting Factory Brooklyn', 'Sony Hall', 'Analog BKNY',
       'Iridium Jazz Club', 'The Bowery Electric', 'Spirit of New York',
       'The Lost Horizon', 'Brooklyn Bowl', 'Apollo Theater', "SOB's",
       'New York City Center - Stage 1', 'The Haunt', 'Elsewhere',
       'The Bug Jar', 'The Chance',
       'Colden Center for Performing Arts at Kupferberg Center for the Arts',
       'Funk N Waffles', 'Kings Hall', 'The Hangar Theatre',
       'Club Helsinki', 'The 9th Ward at Babeville', 'The Bell House',
       'Montage Music Hall', "Lando's", 'Avant Gardner',
       "Lando's Hotel & Lounge", 'Mulcahys', 'Mohawk Place',
       "Sportsmen's Tavern", 'Buffalo Riverworks',
       "Funk 'n Waffles - Downtown", 'Rec Room - Buffalo',
       'Murmrr Theatre', 'Kodak Center for Performing Arts',
       'Market Hotel', 'Smith Opera House', 'Asbury Hall At The Church',
       "Sharkey's Summer Stage",
       'Carnegie Hall - Judy & Arthur Zankel Hall',
       'Asbury Hall inside Babev

In [168]:
# Manually went on Google to find missing venue capacity values
c_df.loc[c_df['venue_name'] == "Brewery Ommegang", 'venue_capacity'] = 5000

In [169]:
c_df.loc[c_df['venue_capacity'] == 0]['venue_name'].unique()

array(['Spirit of New York', "Lando's", "Lando's Hotel & Lounge",
       "Sharkey's Summer Stage", 'Bartlett Theatre in Coxe Hall'],
      dtype=object)